# Deep Learning for Computer Vision with TensorFlow 2.0

Table of contents:
[Lab 0](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/00_test_install.ipynb) | 
[Lab 1](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/01_linear_regression.ipynb) | 
[Lab 2](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/02_tensorflow_logistic_regression.ipynb) | 
[Lab 3a](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/03a_tensorflow_deep_network.ipynb) | 
[Lab 3b](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/03b_deep_mnist_visualize.ipynb) | 
[Lab 4](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/04_mnist_cnn.ipynb) | 
[Lab 5](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/05_data_prep.ipynb) | 
[Lab 6](https://colab.research.google.com/github/embedded-vision/dlcvtf2/blob/master/06_transfer_learning.ipynb) | 

# Lab 5: Data Prep

If you ever want to start over just go to the menu at the top and select Runtime ->  Restart runtime.  (Please do not use Runtime -> Reset all runtimes.)

In this notebook we will delve further into dataset creation. The MNIST dataset used previously was extremly simple and this notebook will give you a better idea of how this is done for typical applications. 


In [ ]:
# Cell 5.1
%tensorflow_version 2.x
import tensorflow as tf
if tf.__version__ != "2.0.0":
    !pip install tensorflow-gpu==2.0.0
    print("Please go to Runtime -> restart runtime and then, once that finishes, rerun this cell.")

from __future__ import absolute_import, division, print_function

import os

from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.transforms as trans

from PIL import Image, ImageEnhance, ImageFilter

%pylab inline

The next cell will download a cat and dog dataset so that the contents can be examined in more detail. The idea is to show how this process is typically completed.

The `base_dir` variable is set to the top of the directories to be examined. 

In [ ]:
# Cell 5.2

zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip", 
                                   cache_dir='.',fname="cats_and_dogs_filtered.zip", extract=True)

base_dir, _ = os.path.splitext(zip_file)
print(base_dir)

Show the directory structure


In [ ]:
# Cell 5.3

# code to show directory structure
print(os.listdir('./datasets'))

print(os.listdir('./datasets/cats_and_dogs_filtered'))

print(os.listdir('./datasets/cats_and_dogs_filtered/train'))

dog_list = os.listdir('./datasets/cats_and_dogs_filtered/train/dogs')
print (dog_list[0:5])
print (len(dog_list))

cat_list = os.listdir('./datasets/cats_and_dogs_filtered/train/cats')
print (cat_list[0:5])
print (len(cat_list))

Explore table of cat pics

In [ ]:
# Cell 5.4

# code to show table of pics from cats

# update offset by values of 9 to see the next set of pictures
offset = 9

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(330 + 1 + i)
    img_file = plt.imread('./datasets/cats_and_dogs_filtered/train/cats/' + cat_list[i+ offset])
    #img_file = plt.resize()
    plt.imshow(img_file)

plt.show()

Explore table of dog pics

In [ ]:
# Cell 5.5

# code to show table of pics from dogs

# update offset by values of 9 to see the next set of pictures
offset = 9

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(330 + 1 + i)
    img_file = plt.imread('./datasets/cats_and_dogs_filtered/train/dogs/' + dog_list[i+offset])
    #img_file = plt.resize()
    plt.imshow(img_file)
plt.show()

The next cell shows how the training and validation directories are setup for the training process. These paths are used to create the datasets that will be used during training and validation. 

In [ ]:
# Cell 5.6

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
print ('Total training cat images:', len(os.listdir(train_cats_dir)))

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
print ('Total training dog images:', len(os.listdir(train_dogs_dir)))

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
print ('Total validation cat images:', len(os.listdir(validation_cats_dir)))

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print ('Total validation dog images:', len(os.listdir(validation_dogs_dir)))

Show function call for the Image Data Generator class and some common arguments

In [ ]:
# Cell 5.7

gen =keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=10,
    zoom_range=0.2,
    channel_shift_range=0.2,
    horizontal_flip=True,
    rescale=1./255)

gimg = plt.imread('./datasets/cats_and_dogs_filtered/train/cats/cat.952.jpg')
ximg = np.expand_dims(gimg,0)

img_iter = gen.flow(ximg)

#aug_imgs=[next(img_iter)[0].astype(np.uint8) for i in range(9)]
aug_imgs=[next(img_iter)[0] for i in range(9)]

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(330 + 1 + i)
    img_file = aug_imgs[i]
    plt.imshow(img_file)
plt.show()

Using the directories provided above two ImageDataGenerators can be created, one for training, the other for validation. These generators will apply augmentations to the images as they are pulled from the training and validation directories. 

In [ ]:
# Cell 5.8

image_size = 160 # All images will be resized to 160x160
batch_size = 9

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),  
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

t_imgs = iter(train_generator)

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(330 + 1 + i)
    img_file = t_imgs[0][0][i]
    plt.imshow(img_file)
plt.show()

print ('Now the validation images')
print ('')
print ('')



v_imgs = iter(validation_generator)

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(330 + 1 + i)
    img_file = v_imgs[0][0][i]
    plt.imshow(img_file)
plt.show()